# 미완성
수정 필요

# 목적
* Grad-CAM

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import time

%matplotlib inline

In [ ]:
print(tf.__version__)
print(tfds.__version__)

## 상수

In [ ]:
BATCH_SIZE = 32
EPOCHS = 1

## 데이터 준비
* imagenet 데이터 중에서 제일 분류 잘 되는 데이터 일부를 가져다 쓰자.

In [ ]:
ds = tfds.load('imagenette/160px-v2')

In [ ]:
X_train, y_train = [], []
for x in ds['train']:
    img = np.array(Image.fromarray(x['image'].numpy()).resize((160,160)))
    X_train.append(img)
    y_train.append(x['label'])
    
X_test, y_test = [], []
for x in ds['validation']:
    img = np.array(Image.fromarray(x['image'].numpy()).resize((160,160)))
    X_test.append(img)
    y_test.append(x['label'])
    
X_train = np.array(X_train)
y_train = np.array(y_train)

X_test = np.array(X_test)
y_test = np.array(y_test)

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

* 데이터 함 그려보자.

In [ ]:
nrows, ncols = 5, 5
fig, ax = plt.subplots(nrows, ncols, figsize=(5,5), dpi=160)
for i in range(nrows):
    for j in range(ncols):
        idx = np.random.randint(X_train.shape[0])
        ax[i][j].imshow(X_train[idx])
        ax[i][j].set_title(str(y_train[idx]), fontsize=10)
        ax[i][j].axis('off')
plt.subplots_adjust(hspace=.5)
plt.show()

In [ ]:
train_dataset = tf.data.Dataset.\
            from_tensor_slices((X_train, y_train)).\
            shuffle(True).batch(BATCH_SIZE)

test_dataset = tf.data.Dataset.\
            from_tensor_slices((X_test, y_test)).batch(BATCH_SIZE)

## 모델 불러오기

In [ ]:
from tensorflow.keras import models, layers, losses, optimizers, metrics

In [ ]:
mobilenetv2 = tf.keras.applications.MobileNetV2(input_shape=(160,160,3), include_top=False, weights='imagenet')

class Model(models.Model):
    def __init__(self, base):
        super(Model, self).__init__()
        self.base = base

    def call(self, inputs):
        x = self.base(inputs)
        x = layers.Flatten()(x)
        x = layers.Dense(10)(x)
        return layers.Activation(tf.nn.softmax)(x)
        
model = Model(mobilenetv2)
model.build((None, 160, 160, 3))

* 사전 학습 된거 좀만 더 트레이닝

In [ ]:
opt = optimizers.Adam(learning_rate=1e-5)
scc = losses.SparseCategoricalCrossentropy()
metric = metrics.SparseCategoricalAccuracy()
loss = metrics.Mean()
acc = metrics.Mean()

In [ ]:
%%time
for e in range(EPOCHS):
    cnt = 0
    for inputs in train_dataset:
        start = time.time()
        _x, _y = inputs
        with tf.GradientTape() as t:
            _y_pred = model(_x)
            _loss = scc(_y, _y_pred)
        grads = t.gradient(_loss, model.trainable_variables)
        opt.apply_gradients(list(zip(grads, model.trainable_variables)))
        loss.update_state(_loss)
        acc.update_state(metric(_y, _y_pred))
        if (cnt+1)%50 == 0:
            print(f"{cnt+1}-th, {time.time() - start:.2f} sec/batch")
        cnt += 1
    print(f"{e+1}/{EPOCHS} loss:{loss.result():.8f} acc:{acc.result()*100:.2f}%")
    loss.reset_states()
    acc.reset_states()